In [2]:
import sqlalchemy as sqla
import pymysql
import pandas as pd
import os

username = "user"  # get this from the instructor
password = "grad5100user"  # get this from the instructor
awsresource = "database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com"  # get this from the instructor
port = 3306

In [3]:
dbname = "topChef"
dburi = f"mysql+pymysql://{username}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)

In [4]:
def run_sql(sql, engine):
    "send sql to database given by engine and return the result"
    with engine.connect() as conn:
        result = conn.execute(sqla.text(sql))
    return result.all()


def df_select(sql, engine):
    "send sql to database given by engine and return the result as a pandas dataframe"
    with engine.connect() as conn:
        result = pd.read_sql(sqla.text(sql), con=conn)
    print(f"Retrieved {result.shape[0]} records")
    return result

In [42]:
from pandas import DataFrame

number_one = run_sql("select season, seasonNumber, name, placement, gender from chefdetails where season not like '%Masters%' and season not like '%Canada%'", engine)
for x in number_one:
    print(x)
df_chefdetails = DataFrame(number_one)

('All Stars: New York', 8, 'Richard Blais', 1, 'Male')
('All Stars: New York', 8, 'Mike Isabella', 2, 'Male')
('All Stars: New York', 8, 'Antonia Lofaso', 3, 'Female')
('All Stars: New York', 8, 'Tiffany Derry', 4, 'Female')
('All Stars: New York', 8, 'Carla Hall', 5, 'Female')
('All Stars: New York', 8, 'Dale Talde', 6, 'Male')
('All Stars: New York', 8, 'Angelo Sosa', 7, 'Male')
('All Stars: New York', 8, 'Fabio Viviani', 8, 'Male')
('All Stars: New York', 8, 'Tre Wilcox', 9, 'Male')
('All Stars: New York', 8, 'Marcel Vigneron', 10, 'Male')
('All Stars: New York', 8, 'Jamie Lauren', 11, 'Female')
('All Stars: New York', 8, 'Tiffani Faison', 11, 'Female')
('All Stars: New York', 8, 'Casey Thompson', 13, 'Female')
('All Stars: New York', 8, 'Spike Mendelsohn', 14, 'Male')
('All Stars: New York', 8, 'Dale Levitski', 15, 'Male')
('All Stars: New York', 8, 'Stephen Asprinio', 15, 'Male')
('All Stars: New York', 8, 'Jennifer Carroll', 17, 'Female')
('All Stars: New York', 8, 'Elia Aboumrad

In [43]:
number_two = run_sql("select season, seasonNumber, episode, challengeType, guestJudge, competedOnTC from judges where season not like '%Masters%' and season not like '%Canada%'", engine)
for x in number_two:
    print(x)
df_judges = DataFrame(number_two)

('All Stars: New York', 8, 1, 'Quickfire', 'Tom Colicchio', None)
('All Stars: New York', 8, 1, 'Elimination', 'Anthony Bourdain', None)
('All Stars: New York', 8, 2, 'Quickfire', 'Joe Jonas', None)
('All Stars: New York', 8, 2, 'Elimination', 'Katie Lee', None)
('All Stars: New York', 8, 3, 'Quickfire', 'David Chang', None)
('All Stars: New York', 8, 3, 'Elimination', 'Anthony Bourdain', None)
('All Stars: New York', 8, 3, 'Elimination', 'Kate Krader', None)
('All Stars: New York', 8, 4, 'Quickfire', 'Tony Mantuano', None)
('All Stars: New York', 8, 4, 'Elimination', 'Tony Mantuano', None)
('All Stars: New York', 8, 5, 'Quickfire', 'Tom Colicchio', None)
('All Stars: New York', 8, 5, 'Elimination', 'Susur Lee', None)
('All Stars: New York', 8, 6, 'Elimination', 'Kerry Heffernan', None)
('All Stars: New York', 8, 7, 'Quickfire', 'Justo Thomas', None)
('All Stars: New York', 8, 7, 'Elimination', 'Ludo Lefebvre', None)
('All Stars: New York', 8, 8, 'Quickfire', 'Isaac Mizrahi', None)
('A

In [11]:
#a-- Male
len(df_chefdetails[df_chefdetails["gender"] == "Male"])

170

In [12]:
#a-- Female
len(df_chefdetails[df_chefdetails["gender"] == "Female"])

152

In [16]:
#b-- Male
len(df_chefdetails[(df_chefdetails["gender"] == "Male") & (df_chefdetails["placement"] == 1)])

14

In [17]:
#b-- Female
len(df_chefdetails[(df_chefdetails["gender"] == "Female") & (df_chefdetails["placement"] == 1)])

6

In [18]:
#c-- Male
len(df_chefdetails[(df_chefdetails["gender"] == "Male") & (df_chefdetails["placement"] <= 3)])

36

In [19]:
#c-- Female
len(df_chefdetails[(df_chefdetails["gender"] == "Female") & (df_chefdetails["placement"] <= 3)])

25

In [25]:
#d
# Assuming every top 3 contestant has a probability of winning of 1/3,
# we expect 25/3=8.33 female victories across the 20 seasons.
# This number is lower than the true value of 6, but is it significantly lower?
# To figure this out, we use the binomial distribution

from scipy.stats import binom

binomial_dist = binom(25, 1/3)
binomial_dist.cdf(6)

# since the output is above 0.05, we find that though there are fewer than expected female victories, the difference is not significant. 


0.22153925453500153

In [44]:
#4
number_four = run_sql("select name, season, case when guestJudge is null then 'No' else guestJudge end as guest_judge from (select distinct cd.name as name, cd.season as season, j.competedOnTC as guestJudge from chefdetails cd left join judges j on j.guestJudge = cd.name where cd.placement = 1 and cd.season not like '%Masters%' and cd.season not like '%Canada%') t1", engine)
for x in number_four:
    print(x)
df_guestJudges = DataFrame(number_four)

('Richard Blais', 'All Stars: New York', 'Yes')
('Melissa King', 'All-Stars L.A.', 'Yes')
('Mei Lin', 'Boston', 'No')
('Jeremy Ford', 'California', 'No')
('Brooke Williamson', 'Charleston', 'Yes')
('Stephanie Izard', 'Chicago', 'Yes')
('Kevin Sbraga', 'D.C.', 'No')
('Joseph Flamm', 'Colorado', 'No')
('Kah-wai Lo', 'Houston', 'No')
('Kelsey Barnard Clark', 'Kentucky', 'Yes')
('Michael Voltaggio', 'Las Vegas', 'Yes')
('Ilan Hall', 'Los Angeles', 'Yes')
('Hung Huynh', 'Miami', 'Yes')
('Nicholas Elmi', 'New Orleans', 'No')
('Hosea Rosenberg', 'New York', 'Yes')
('Gabe Erales', 'Portland', 'No')
('Harold Dieterle', 'San Francisco', 'Yes')
('Kristen Kish', 'Seattle', 'Yes')
('Paul Qui', 'Texas', 'No')
('Kah-wai Lo', 'World All Stars', 'No')


In [48]:
#5
number_five = run_sql("select distinct cd.name as name, cd.season as season from chefdetails cd left join judges j on j.guestJudge = cd.name where cd.placement != 1 and j.competedOnTC = 'Yes' and cd.season not like '%Masters%' and cd.season not like '%Canada%'", engine)
for x in number_five:
    print(x)
df_nonWinner = DataFrame(number_five)

('Mike Isabella', 'All Stars: New York')
('Antonia Lofaso', 'All Stars: New York')
('Tiffany Derry', 'All Stars: New York')
('Dale Talde', 'All Stars: New York')
('Spike Mendelsohn', 'All Stars: New York')
('Stephen Asprinio', 'All Stars: New York')
('Bryan Voltaggio', 'All-Stars L.A.')
('Gregory Gourdet', 'All-Stars L.A.')
('Karen Akunowicz', 'All-Stars L.A.')
('Lee Anne Wong', 'All-Stars L.A.')
('Nini Nguyen', 'All-Stars L.A.')
('Gregory Gourdet', 'Boston')
('Melissa King', 'Boston')
('Amar Santana', 'California')
('Kwame Onwuachi', 'California')
('Karen Akunowicz', 'California')
('Sheldon Simeon', 'Charleston')
('Sam Talbot', 'Charleston')
('Richard Blais', 'Chicago')
('Antonia Lofaso', 'Chicago')
('Spike Mendelsohn', 'Chicago')
('Dale Talde', 'Chicago')
('Tiffany Derry', 'D.C.')
('Adrienne Cheatham', 'Colorado')
('Carrie Baird', 'Colorado')
('Lee Anne Wong', 'Colorado')
('Nini Nguyen', 'Kentucky')
('Bryan Voltaggio', 'Las Vegas')
('Mike Isabella', 'Las Vegas')
('Sam Talbot', 'Los A